In [ ]:
// purpose of this notebook:
// to find coins that have the highest likelihood of going up with the least amount of new capital inflows

// data we need:
// 1. total coin market cap ovet time
// 2. a list of coins
// 3. the market cap of those coins over

// what we do with that data:
// * convert the coin market caps of each coin into a dominance %
// * calculate the time since the last the coin last hit peak dominance
// * multiply the dominance % by the time since the last peak
// * sort the coins in descending order of the product of the dominance % and the time since the last peak

#r "nuget: FSharp.Data"
#r "nuget: Microsoft.Data.Analysis"

open FSharp.Data

type BinanceExchangeInfo = JsonProvider<"https://api.binance.com/api/v3/exchangeInfo">
type PriceStream = JsonProvider<"https://api.binance.com/api/v3/klines?symbol=1INCHDOWNUSDT&interval=1d">
type CoinGecko = JsonProvider<"https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=250&page=1&sparkline=false">

let show a =
    printf "%A\n" a

type MarketCap = 
    { Symbol: string
    ; MarketCap: int64
    ; Rank: int }

let geckoByPage (page:int) = 
    CoinGecko.Load("https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=250&page=" + page.ToString() + "&sparkline=false")

let geckoSubByPage page =
    geckoByPage page
    |> Array.map (fun x -> { Symbol = x.Symbol; MarketCap = x.MarketCap; Rank = x.MarketCapRank })

let geckoSub =
    [|1..10|]
    |> Array.fold (fun acc x -> Array.append acc (geckoSubByPage x)) Array.empty
    // (geckoSubByPage 1)
    // |> Array.append (geckoSubByPage 2)

show geckoSub.Length

let exchInfo = BinanceExchangeInfo.Load("https://api.binance.com/api/v3/exchangeInfo")

let assets = 
    exchInfo.Symbols 
    |> Array.filter (fun x-> x.QuoteAsset.IndexOf("USDT") <> -1 )
    |> Array.filter (fun x-> x.BaseAsset.IndexOf("UP") = -1 && x.BaseAsset.IndexOf("DOWN") = -1) 
    |> Array.map (fun x -> x.Symbol) 
    |> Array.sort
    //|> Array.take 20

show assets.Length

type Candle = 
    { Time : uint64
    ; Open : decimal
    ; High:  decimal
    ; Low : decimal
    ; Close: decimal }
    
let makeCandle (assetRow:(decimal array)) =
    { Time = (uint64)assetRow.[0]
    ; Open = assetRow.[1]
    ; High = assetRow.[2]
    ; Low = assetRow.[3]
    ; Close = assetRow.[4] }

let assetHistory symbol (limit:uint64) =
    PriceStream.Load("https://api.binance.com/api/v3/klines?symbol=" + symbol + "&interval=1d&limit=" + limit.ToString())
    |> Array.map makeCandle

let ethHistory = 
    assetHistory "ETHUSDT" 1000UL

let ethPricedAssetHistory limit symbol = 
    let asset = assetHistory symbol limit
    let eth = 
        ethHistory
        |> Array.skip (ethHistory.Length - asset.Length)

    asset
    |> Array.zip eth
    |> Array.map (fun (ethRow, assetRow) -> 
        { Time = assetRow.Time
        ; Open = assetRow.Open / ethRow.Open
        ; High = assetRow.High / ethRow.High
        ; Low = assetRow.Low / ethRow.Low
        ; Close = assetRow.Close / ethRow.Close })

let ath assetHistory = 
    assetHistory 
    |> Array.maxBy (fun x -> x.High)

let timeSinceLastAth now assetHistory =
    let lastAth = (ath assetHistory).Time
    now - lastAth

let daysSinceLastAth now assetHistory =
    (timeSinceLastAth now assetHistory) / (24UL * 60UL * 60UL) / 1000UL

"BTCUSDT" |> (ethPricedAssetHistory 1000UL) |> (daysSinceLastAth ethHistory.[ethHistory.Length - 1].Time)

type assetAthDays = 
    { Asset: string
    ; MarketCap: int64
    ; Rank: int
    ; DaysSinceLastAth: uint64
    ; Order: float }

let allDaysSinceLastAth limit assetList = 
    assetList
    |> Array.Parallel.map (fun (asset:string) -> 
        show asset

        let ast = asset.Replace("USDT", "").ToLowerInvariant()
        
        let cap = 
            geckoSub 
            |> Array.filter (fun x -> x.Symbol = ast.ToLowerInvariant())
            |> Array.sumBy (fun x -> x.MarketCap)

        let rank = 
            geckoSub 
            |> Array.filter (fun x -> x.Symbol = ast.ToLowerInvariant())
            |> Array.map (fun x -> x.Rank)
            |> Array.max 

        let dsath = 
            asset 
            |> ethPricedAssetHistory limit 
            |> daysSinceLastAth ethHistory.[ethHistory.Length - 1].Time

        let order = 
            // ((float(dsath) + 1.0) * Math.Sqrt(float(cap) + 1.0))
            (float(dsath) + 1.0) / float(rank) 

        { Asset = ast
        ; MarketCap = cap / 1000000L
        ; Rank = rank
        ; DaysSinceLastAth = dsath
        ; Order = order })

assets  
|> Array.take assets.Length
|> allDaysSinceLastAth 1000UL
|> Array.filter (fun x -> x.MarketCap <> 0L)
|> Array.sortBy (fun x -> x.Order)
//|> Array.skip 20

Installed Packages FSharp.Data, 4.2.3 Microsoft.Data.Analysis, 0.18.0